In [3]:
from openai import AsyncAzureOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel
from agents.run import RunConfig
import os
from dotenv import load_dotenv
from pathlib import Path
import logging

In [6]:
# Get the absolute path to the .env file
current_dir = Path().absolute()
env_path = current_dir / ".env"
print(f"Looking for .env file at: {env_path}")
print(f"Does .env file exist? {env_path.exists()}")

# Load environment variables from .env file with explicit path
load_dotenv(dotenv_path=env_path)

# Debug environment variables to ensure they're loaded
print(f"API KEY: {os.getenv('AZURE_OPENAI_API_KEY', '')[:5] if os.getenv('AZURE_OPENAI_API_KEY') else 'Not found'}...")
print(f"API VERSION: {os.getenv('AZURE_OPENAI_API_VERSION', 'Not found')}")
print(f"ENDPOINT: {os.getenv('AZURE_OPENAI_ENDPOINT', 'Not found')}")
print(f"DEPLOYMENT: {os.getenv('AZURE_OPENAI_DEPLOYMENT', 'Not found')}")

Looking for .env file at: /home/chicha/repos/openai-sdk-101/01_hello_agent/.env
Does .env file exist? True
API KEY: 2geV8...
API VERSION: 2025-01-01-preview
ENDPOINT: https://open-aj.openai.azure.com
DEPLOYMENT: gpt-4o


In [7]:
# Configure logging to disable verbose HTTP request logs from OpenAI client
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)

try:
    azure_openai_client = AsyncAzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY", ""),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION", ""),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", ""),
        azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT", ""),
    )
    print("Azure OpenAI client created successfully")
except Exception as e:
    print(f"Error creating Azure OpenAI client: {e}")

Azure OpenAI client created successfully


In [8]:
# Test if the client works with a simple completion
try:
    response = await azure_openai_client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT", ""),
        messages=[{"role": "user", "content": "Say hello"}]
    )
    print("Client test successful:")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Error testing Azure OpenAI client: {e}")

Client test successful:
Hello! 😊 How can I assist you today?


In [9]:
try:
    model = OpenAIChatCompletionsModel(model="gpt-4o", openai_client=azure_openai_client)
    print("Model created successfully")
    config = RunConfig(
        model=model,
        tracing_disabled=True,
    )
    print("Config created successfully")
except Exception as e:
    print(f"Error creating model or config: {e}")

Model created successfully
Config created successfully


In [11]:
try:
    agent = Agent(
        name="Master Agent",
        model=model,
    )
    print("Agent created successfully")
except Exception as e:
    print(f"Error creating agent: {e}")

Agent created successfully


In [ ]:
try:
    # Use the async version instead of run_sync to avoid event loop conflicts
    async def run_agent():
        result = await Runner.run(agent, "Hello, how are you.", run_config=config)
        print("\nCALLING AGENT\n")
        print(result.final_output)
        return result

    # Execute the async function directly - Jupyter will handle it properly
    result = await run_agent()
except Exception as e:
    import traceback

    print(f"Error running agent: {e}")
    print(traceback.format_exc())


CALLING AGENT

Hello! I'm just a virtual assistant, so I don't have feelings, but thank you for asking! How about you? How can I assist you today? 😊
